# Sample Document Generator 

This is the first program of my project which will generate the sample set of documents the main "Retention Checker" program will check. Because confidentiality protocols prevent me from using real document information, I need dummy documents to run the code on. 

The goal of this program is to produce an outfile of a large number of dummy documents. The bank I work for has over 15,000 loans, each with dozens of documents but for the purposes of this project, I'd like the main program to check a sample set of 500 documents. Considering how specific the parameters for retention are, I feel like I need a lot of samples to see the code really working. 

## Setting Up the Program 

In order to generate the sample docs I need to do some set-up. First, I need to import a few modules I will be using later.

In [ ]:
import random
from random import randint
import datetime
import csv

Next, I need to set up my reference dates. 

In [3]:
current_date = datetime.datetime.today()
open_date = datetime.datetime(2001,1,1)

print("current date is", current_date)
print("open date is", open_date)

current date is 2020-05-09 21:11:04.455519
open date is 2001-01-01 00:00:00


Using the datetime module I set up the current date. In reality, new documents will be produced every day so to simulate that, I need the program to operate using the current date. 

I also set up a "Open Date" which represents the date my hypothetical bank opened. No documents can be created before that date which gets me as close to reality as possible with the dummy documents. 

Finally, I import my "Reference Schedule Guide."

In [ ]:
reference = open("Retention Schedule Guide", 'r') 
doc_type_list = reference.readlines()
reference.close()

This step isn't entirely necessary. I created the guide for my personal reference when setting up the retention parameters. However, I figured I could easily extract the Document Type Codes from the guide to save some typing time. 

In [ ]:
doc_series_codes = [] 
for line in doc_type_list[1:]: 
    doc_line = line.split()
    doc_type = doc_line[0] 
    doc_series_codes.append(doc_type)

Using a for loop and accumulator starting with position [1] of the guide in order to skip the header text. Then I slice out the Doc Code in position [0] of each line and add it to the accumulator. 

In [ ]:
print(doc_series_codes)
['ACC100', 'ACC180', 'ACC160', 'ACC300', 'ACC340', 'ACC400', 'ACC500', 'ACC520', 'ADM440', 'BNK100', 'BNK120', 'BNK130', 'BNK140', 'BNK160', 'BNK170', 'BNK180', 'BNK300', 'BNK400', 'GEN200', 'GEN210']


I now have a list of the potential Document Type Codes to generate sample documents from. 